# Setup 
install Python 3.10 and then install VALL-E

In [ ]:
!wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
!bash ./py310.sh -b -f -p /usr/local
!python -m ipykernel install --name "py310" --user

In [ ]:
%cd vall-e
!pip install -e .
%cd ~/../notebooks

# Setup data

In [ ]:
!git clone https://github.com/enhuiz/vall-e
!wget https://dl.fbaipublicfiles.com/librilight/data/librispeech_finetuning.tgz
!unzip librispeech_finetuning.tgz

In [ ]:
import os

## setup Libri-Light

!mkdir vall-e/data/libri
import os

for k in os.listdir('librispeech_finetuning/9h/clean'):
    for j in os.listdir('librispeech_finetuning/9h/'+'clean'):
        for z in os.listdir('librispeech_finetuning/9h/clean/'+j):
            for i in os.listdir('librispeech_finetuning/9h/clean/'+j+'/'+z):
                os.rename('librispeech_finetuning/9h/clean/'+j+'/'+z+'/'+i, f'vall-e/data/libri/{i}')
        
    break
lst = []
for i in os.listdir('vall-e/data/libri'):
    try:
        if 'trans' in i:
            with open(f'vall-e/data/libri/{i}') as text_file:
                for row in text_file:
                    z = row.split('-')
                    name = z[0]+'-'+z[1]+ '-' + z[2].split(' ')[0]
                    text = " ".join(z[2].split(' ')[1:])
                    # print(name, text)
                    lst.append([name, text])

                        
    except:
        None      

for i in lst:
    try:
        with open('vall-e/data/libri/'+i[0]+'.txt', 'x') as file:
            # print(i[1])
            file.write(i[1])
    except:
        with open('vall-e/data/libri/'+i[0]+'.txt', 'w+') as file:
            # print(i[1])
            file.write(i[1])

for i in sorted(os.listdir('vall-e/data/libri')):
    if i.split('.')[1] == 'txt':
        # print(i)
        print('.'.join([i.split('.')[0],'normalized',i.split('.')[1]]))
        os.rename('vall-e/data/libri/'+i, 'vall-e/data/libri/'+'.'.join([i.split('.')[0],'normalized',i.split('.')[1]]))

In [ ]:
!pip install AudioConverter
!audioconvert convert vall-e/data/libri/ vall-e/data/libri --output-format .wav


In [ ]:
%cd ~/../notebooks
!mv config/libri vall-e/config/libri
!mv config/libri2 vall-e/config/libri2


In [ ]:
%cd vall-e
!python -m vall_e.emb.qnt data/libri
!python -m vall_e.emb.g2p data/libri

# Setting up your own data


Use the snippet below to move the data for the 248-130644 text files into a new folder,  vall-e/data/libri2. Then, on your local machine, using your preferred audio recording application, record yourself reading each of the relevant text lines. Save each audio file in .wav format with the corresponding title. Then upload them to the Gradient Notebook directory, vall-e/data/libri2.

In [ ]:
!mkdir vall-e/data/libri2

In [ ]:
import os
for i in sorted(os.listdir('vall-e/data/libri')):
    if i.split('.')[1] == 'normalized' and i.split('-')[0] == '248' and i.split('-')[1] == '130644':
        # print(i)
        os.rename('vall-e/data/libri/'+i, 'vall-e/data/libri2/'+i)

In [ ]:
%cd vall-e
!python -m vall_e.emb.qnt data/libri2
!python -m vall_e.emb.g2p data/libri2

# Train

In [ ]:
%cd vall-e

# run on Libri Light multi-GPU
python -m torch.distributed.launch --nproc_per_node 4 -m vall_e.train yaml=config/libri/ar.yml
python -m torch.distributed.launch --nproc_per_node 4 -m vall_e.train yaml=config/libri/nar.yml

# run on Libri Light single-GPU

python -m vall_e.train yaml=config/libri/ar.yml
python -m vall_e.train yaml=config/libri/nar.yml

# run on your sample of Libri Light rerecording
# Multi-GPU

python -m torch.distributed.launch --nproc_per_node 4 -m vall_e.train yaml=config/libri2/ar.yml
python -m torch.distributed.launch --nproc_per_node 4 -m vall_e.train yaml=config/libri2/nar.yml

# Single GPU
python -m vall_e.train yaml=config/libri2/ar.yml
python -m vall_e.train yaml=config/libri2/nar.yml



In [ ]:
%cd vall-e/
# export libri light model
!python -m vall_e.export zoo/ar.pt yaml=config/libri/ar.yml
!python -m vall_e.export zoo/nar.pt yaml=config/libri/nar.yml

# export your sample models
# !python -m vall_e.export zoo/ar.pt yaml=config/libri2/ar.yml
# !python -m vall_e.export zoo/nar.pt yaml=config/libri2/nar.yml


# Sample the model

Create a new audio sample using the trained model in the target voice

In [ ]:
%cd vall-e/
# your voice
!python -m vall_e "BUT WOULD MAKE THE VERY BEST OF THE CHANCES THAT LIFE OFFERED HER AND CONQUER CIRCUMSTANCES BY HER EXCEPTIONAL CLEVERNESS" data/libri2/248-130644-0018.wav toy1.wav
# libri light
# !python -m vall_e "BUT WOULD MAKE THE VERY BEST OF THE CHANCES THAT LIFE OFFERED HER AND CONQUER CIRCUMSTANCES BY HER EXCEPTIONAL CLEVERNESS" data/libri/248-130644-0018.wav toy1.wav


In [29]:
from IPython.display import Audio
Audio('toy1.wav')

